# TGRAINS Crop Model Viz interface shim

A `cppyy` example of using the crop model, which we currently need to shim as pass-by-reference is not supported for `std::vector` in `cppyy`.

In [1]:
%cd ..

/vagrant_data


In [2]:
import cppyy
import ctypes
from array import array
from ctypes import c_double
from ctypes import c_int
from cppyy.gbl.std import vector

(Re-)building pre-compiled headers (options: -O2 -mavx); this may take a minute ...


In [3]:
cppyy.include('model/Landscape2019_LinuxSO.h')
cppyy.load_library('model/libLandscape2019_LinuxSO.so')

Define a shim in C++. Dynamically compiled at runtime by cppyy using the cling interpreter:

In [4]:
cppyy.cppdef("""
    #include "model/Landscape2019_LinuxSO.h"
    
    typedef struct cropData
    {
        int myUniqueLandscapeID;
        double maxCropArea;
        std::vector<double> cropAreas;
        std::vector<int> livestockNumbers;
        
        double greenhouseGasEmissions;
        double nLeach;
        double profit;
        double production;
        
        std::vector<double> pesticideImpacts;
        std::vector<double> nutritionaldelivery;
        std::vector<double> healthRiskFactors;
        int errorFlag;
    } cropData;
    
    cropData initTGRAINS(int myUniqueLandscapeID) 
    {
        cropData d = {
            myUniqueLandscapeID, 
            0.0,
            std::vector<double>(),
            std::vector<int>(),
            
            0.0,
            0.0,
            0.0,
            0.0,
            
            std::vector<double>(),
            std::vector<double>(),
            std::vector<double>(),
            0
        };
        
        initialiseTGRAINS_RLM(
            d.myUniqueLandscapeID,
            d.maxCropArea,
            d.cropAreas,
            d.livestockNumbers,
            d.errorFlag
        );
        
        return d;
    };
    
    void runTGRAINS(cropData& myCropData) 
    {
        runTGRAINS_RLM(
            myCropData.cropAreas,
            myCropData.livestockNumbers,
            myCropData.greenhouseGasEmissions, 
            myCropData.nLeach,
            myCropData.pesticideImpacts,
            myCropData.profit,
            myCropData.production,
            myCropData.nutritionaldelivery,
            myCropData.healthRiskFactors,
            myCropData.errorFlag
        );
    };
""")

True

Now, we can use it easily from python:

In [5]:
list(cppyy.gbl.getLandscapeIDs())

[101, 102]

In [6]:
cropData = cppyy.gbl.initTGRAINS(101)

In [7]:
cropData

In [8]:
cropData.maxCropArea

60000.0

In [9]:
list(cropData.cropAreas)

[400.0, 400.0, 200.0, 800.0, 800.0, 400.0, 400.0, 200.0, 800.0, 800.0]

In [10]:
list(cropData.livestockNumbers)

[1000, 2000, 3000]

Run crop model passing the struct which was initialised above:

In [11]:
cppyy.gbl.runTGRAINS(cropData)

In [12]:
list(cropData.pesticideImpacts)

[0.25384615384615383,
 1.2692307692307692,
 0.7615384615384615,
 0.38076923076923075,
 0.45692307692307693]

In [13]:
cropData.profit

122.53846153846153

In [14]:
{ k:v for k,v in vars(cppyy.gbl.cropData).items() if not k.startswith('_') }.keys()

dict_keys(['myUniqueLandscapeID', 'maxCropArea', 'cropAreas', 'livestockNumbers', 'greenhouseGasEmissions', 'nLeach', 'profit', 'production', 'pesticideImpacts', 'nutritionaldelivery', 'healthRiskFactors', 'errorFlag'])

In [15]:
cppyy.gbl.getLandscapeString(101)

'East Anglia'

In [16]:
cppyy.gbl.getCropString(0)

'Wheat'